In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

## Build Linear Regression model based on the data

In [3]:
data=pd.read_csv("data_encoded_IS.csv")
data.head(10)

,id,city,price,year,manufacturer,make,condition,cylinders,fuel,odometer,...,type_onehot_11,type_onehot_12,type_onehot_13,type_onehot_14,drive_onehot_1,drive_onehot_2,drive_onehot_3,drive_onehot_4,transmission_encoded,fuel_encoded
0,0,"abilene, TX",9000,2009,chevrolet,suburban lt2,good,8,gas,217743.0,...,0,0,0,0,0,0,0,1,0,2
1,1,"abilene, TX",31999,2012,ram,2500,NaN,0,diesel,NaN,...,0,0,0,0,0,0,1,0,0,0
2,2,"abilene, TX",16990,2003,ram,3500,NaN,0,diesel,NaN,...,0,0,0,0,0,0,1,0,1,0
3,3,"abilene, TX",6000,2002,gmc,sierra 1500,good,8,gas,195000.0,...,0,0,0,0,1,0,0,0,0,2
4,4,"abilene, TX",37000,2012,chevrolet,3500,excellent,8,diesel,178000.0,...,0,0,0,0,1,0,0,0,0,0
5,5,"abilene, TX",3700,2003,NaN,F150,fair,8,gas,269000.0,...,0,0,0,0,1,0,0,0,0,2
6,6,"abilene, TX",19950,2013,ford,f-250,NaN,8,gas,116792.0,...,0,0,0,0,1,0,0,0,0,2
7,9,"abilene, TX",25950,2015,ford,f-350,NaN,8,gas,121030.0,...,0,1,0,0,1,0,0,0,0,2
8,11,"abilene, TX",9700,2010,cadillac,srx luxury collection,good,8,gas,140000.0,...,0,0,0,0,0,1,0,0,0,2
9,12,"abilene, TX",2500,2001,chevrolet,silverado 1500,fair,8,gas,220000.0,...,0,0,0,0,0,0,0,1,0,2


In [33]:
print(data.columns)

Index(['id', 'city', 'price', 'year', 'manufacturer', 'make', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'type', 'lat', 'long', 'state', 'predicted_state',
       'state_comb', 'income_rank', 'median_income', 'condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'title_status_onehot_1',
       'title_status_onehot_2', 'title_status_onehot_3',
       'title_status_onehot_4', 'title_status_onehot_5',
       'title_status_onehot_6', 'title_status_onehot_7', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
     

In [73]:
train_proportion = 0.8
features_colname=['year','odometer','income_rank','condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'title_status_onehot_1',
       'title_status_onehot_2', 'title_status_onehot_3',
       'title_status_onehot_4', 'title_status_onehot_5',
       'title_status_onehot_6', 'title_status_onehot_7', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
       'type_onehot_10', 'type_onehot_11', 'type_onehot_12', 'type_onehot_13',
       'type_onehot_14', 'drive_onehot_1', 'drive_onehot_2', 'drive_onehot_3',
       'drive_onehot_4', 'transmission_encoded', 'fuel_encoded']
data_clean=data[features_colname+["price"]]

# #fill nans in features
# features=features.fillna(features.mean())

#  drop nan
data_clean=data_clean.dropna(axis = 0, how ='any')
print(len(data_clean))

features=data_clean[features_colname]
features['odometer']=features['odometer']/features['odometer'].mean()
features['income_rank']=features['income_rank']/features['income_rank'].mean()
target=data_clean["price"]

index_list=[i for i in range(len(data_clean))]

random.shuffle(index_list)
train_list=index_list[:round(len(data_clean)*train_proportion)]
test_list=index_list[round(len(data_clean)*train_proportion):]

# the following variable records the features of examples in the training set
train_x=features.iloc[train_list]
# the following variable records the features of examples in the test set
test_x=features.iloc[test_list]
# the following variable records the labels of examples in the training set
train_y=target.iloc[train_list]
# the following variable records the labels of examples in the test set
test_y=target.iloc[test_list]
print(len(train_x))
print(len(test_x))

344197


/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


275358
68839


In [74]:
features.head(10)

,year,odometer,income_rank,condition_onehot_1,condition_onehot_2,condition_onehot_3,condition_onehot_4,condition_onehot_5,condition_onehot_6,condition_onehot_7,...,type_onehot_11,type_onehot_12,type_onehot_13,type_onehot_14,drive_onehot_1,drive_onehot_2,drive_onehot_3,drive_onehot_4,transmission_encoded,fuel_encoded
0,2009,2.082412,0.948878,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
3,2002,1.864907,0.948878,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
4,2012,1.702325,0.948878,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,2003,2.572615,0.948878,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
6,2013,1.116955,0.948878,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,2
7,2015,1.157485,0.948878,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,2
8,2010,1.338907,0.948878,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
9,2001,2.103997,0.948878,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
10,2016,1.424980,0.948878,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
11,2011,0.717272,0.948878,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2


In [39]:
def MAE_relative(y,pred):# assume input as arrays
    return abs(1-pred/y).mean()

In [76]:
Xtrain=np.array(train_x)
Xtest=np.array(test_x)
Ytrain=np.array(train_y)
Ytest=np.array(test_y)

model=LinearRegression(fit_intercept=True)
model.fit(Xtrain,Ytrain)
w_coef=np.array(model.coef_)

train_pred = model.predict(Xtrain)
test_pred = model.predict(Xtest)

train_MAE_relative = MAE_relative(Ytrain,train_pred)
test_MAE_relative = MAE_relative(Ytest,test_pred)

print("Train relative error:",train_MAE_relative)
print("Test relative error:",test_MAE_relative)

Train relative error: 0.615088647191849
Test relative error: 0.6097119983173068


In [77]:
print(w_coef)

[  846.82637575 -4508.50128259   191.94119253  -921.96156359
 -1879.75491465 -1847.9655821    521.32525893  6880.96129968
   901.32090966 -3653.92540794 -4204.29320421 -9140.76444603
 -7584.71986974 -7334.34241116 -4955.561028   -2815.35398118
  2051.98035672 33983.0545836   1019.84507122  2659.13393956
  2157.223271    2607.0794485  -3568.46277779 -2813.57423796
 -2061.24471453 -1905.31678831   -58.85912633  2773.44198072
  1637.24594013 -2956.40561779 -1258.22082861  -974.7666628
  3539.9742953   1088.84227692  3207.58211056 -2986.65326882
  2030.88438829  -877.18467707 -3260.56402218  2729.50666709
 -2656.83607992  -633.46846889   560.79788172   668.0534963
 -2361.41380921]


In [82]:
model_ridge=Ridge(alpha=1,fit_intercept=True)
model_ridge.fit(Xtrain,Ytrain)
w_coef=np.array(model_ridge.coef_)

train_pred = model_ridge.predict(Xtrain)
test_pred = model_ridge.predict(Xtest)

train_MAE_relative = MAE_relative(Ytrain,train_pred)
test_MAE_relative = MAE_relative(Ytest,test_pred)

print("Train relative error:",train_MAE_relative)
print("Test relative error:",test_MAE_relative)

Train relative error: 0.6150786059185714
Test relative error: 0.6097025459208245


In [83]:
w_coef

array([  846.80643862, -4508.6857908 ,   192.02972776,  -921.81563954,
       -1879.32311234, -1847.75597486,   521.4305815 ,  6871.91573309,
         901.57732655, -3646.02891447, -4174.66792233, -9076.9201168 ,
       -7554.9477161 , -7300.74509118, -4925.95313844, -2785.82491013,
        2079.07409652, 33739.98479674,  1006.4707076 ,  2644.63024049,
        2126.61021991,  2592.44244247, -3468.92598492, -2826.66326792,
       -2074.56435748, -1904.94074095,   -58.2712625 ,  2774.09020402,
        1638.05389015, -2955.71985716, -1257.24965129,  -974.31826528,
        3532.60591501,  1089.21711679,  3207.76326827, -2985.99061696,
        2031.13713549,  -876.58650273, -3259.79063328,  2729.65763312,
       -2657.21686972,  -633.58114283,   561.14038051,   667.98977869,
       -2361.42907109])

In [84]:
print(model_ridge.intercept_)

-1674243.8347762576


In [5]:
data[['condition','condition_onehot_1','condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4']].head(30)

,condition,condition_onehot_1,condition_onehot_2,condition_onehot_3,condition_onehot_4
0,good,0,0,1,0
1,NaN,0,0,0,0
2,NaN,0,0,0,0
3,good,0,0,1,0
4,excellent,1,0,0,0
5,fair,0,1,0,0
6,NaN,0,0,0,0
7,NaN,0,0,0,0
8,good,0,0,1,0
9,fair,0,1,0,0


## use the data set with grouping

In [4]:
data_grouping=pd.read_csv("data_with_grouping_operations_IS.csv")
data_grouping.head(10)

,id,city,price,year,manufacturer,make,condition,cylinders,fuel,odometer,...,title_status_price_std,transmission_mean_price,transmission_median_price,transmission_price_std,type_mean_price,type_median_price,type_price_std,manufacturer_make_mean_price,manufacturer_make_median_price,manufacturer_make_price_std
0,0,"abilene, TX",9000,2009,chevrolet,suburban lt2,good,8,gas,217743.0,...,11785.78595,13730.855095,10500.0,11631.594162,13657.134365,11490.0,10164.477117,9000.000000,9000.0,NaN
1,1,"abilene, TX",31999,2012,ram,2500,NaN,0,diesel,NaN,...,11785.78595,13730.855095,10500.0,11631.594162,NaN,NaN,NaN,25723.333243,26500.0,12444.212024
2,2,"abilene, TX",16990,2003,ram,3500,NaN,0,diesel,NaN,...,11785.78595,11948.546827,7995.0,12972.814921,NaN,NaN,NaN,29935.459606,29500.0,14389.268281
3,3,"abilene, TX",6000,2002,gmc,sierra 1500,good,8,gas,195000.0,...,11785.78595,13730.855095,10500.0,11631.594162,20970.623623,19988.0,12565.326508,19234.861502,17987.5,11444.907291
4,4,"abilene, TX",37000,2012,chevrolet,3500,excellent,8,diesel,178000.0,...,11785.78595,13730.855095,10500.0,11631.594162,20970.623623,19988.0,12565.326508,18125.561404,11900.0,14429.134264
5,5,"abilene, TX",3700,2003,NaN,F150,fair,8,gas,269000.0,...,11785.78595,13730.855095,10500.0,11631.594162,20970.623623,19988.0,12565.326508,NaN,NaN,NaN
6,6,"abilene, TX",19950,2013,ford,f-250,NaN,8,gas,116792.0,...,11785.78595,13730.855095,10500.0,11631.594162,20970.623623,19988.0,12565.326508,20877.369089,18000.0,13317.259116
7,9,"abilene, TX",25950,2015,ford,f-350,NaN,8,gas,121030.0,...,11785.78595,13730.855095,10500.0,11631.594162,19212.896847,16999.0,13365.617749,23275.038261,19999.0,13133.456744
8,11,"abilene, TX",9700,2010,cadillac,srx luxury collection,good,8,gas,140000.0,...,11785.78595,13730.855095,10500.0,11631.594162,13657.134365,11490.0,10164.477117,13218.400000,12500.0,5615.091622
9,12,"abilene, TX",2500,2001,chevrolet,silverado 1500,fair,8,gas,220000.0,...,11785.78595,13730.855095,10500.0,11631.594162,20970.623623,19988.0,12565.326508,18838.962029,17197.0,11442.627776


In [6]:
print(data_grouping.columns)

Index(['id', 'city', 'price', 'year', 'manufacturer', 'make', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'type', 'lat', 'long', 'state', 'predicted_state',
       'state_comb', 'income_rank', 'median_income', 'condition_onehot_1',
       'condition_onehot_2', 'condition_onehot_3', 'condition_onehot_4',
       'condition_onehot_5', 'condition_onehot_6', 'condition_onehot_7',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'title_status_onehot_1',
       'title_status_onehot_2', 'title_status_onehot_3',
       'title_status_onehot_4', 'title_status_onehot_5',
       'title_status_onehot_6', 'title_status_onehot_7', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
     

In [36]:
train_proportion = 0.8
features_colname=['year','odometer','income_rank','median_income',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
       'type_onehot_10', 'type_onehot_11', 'type_onehot_12', 'type_onehot_13',
       'type_onehot_14','drive_onehot_1', 'drive_onehot_2', 'drive_onehot_3',
       'drive_onehot_4', 'title_status_mean_price','manufacturer_mean_price','manufacturer_make_mean_price','transmission_encoded', 'fuel_encoded']
data_clean=data_grouping[features_colname+["price"]]

# #fill nans in features
# features=features.fillna(features.mean())

#  drop nan
data_clean=data_clean.dropna(axis = 0, how ='any')
print(len(data_clean))
features=data_clean[features_colname]

# scaling
features['odometer']=(features['odometer']-features['odometer'].mean())/features['odometer'].std()
features['median_income']=(features['median_income']-features['median_income'].mean())/features['median_income'].std()
features['manufacturer_mean_price']=(features['manufacturer_mean_price']-features['manufacturer_mean_price'].mean())/features['manufacturer_mean_price'].std()
features['manufacturer_make_mean_price']=(features['manufacturer_make_mean_price']-features['manufacturer_make_mean_price'].mean())/features['manufacturer_make_mean_price'].std()
features['income_rank']=features['income_rank']/features['income_rank'].max()
target=data_clean["price"]

index_list=[i for i in range(len(data_clean))]

random.shuffle(index_list)
train_list=index_list[:round(len(data_clean)*train_proportion)]
test_list=index_list[round(len(data_clean)*train_proportion):]

# the following variable records the features of examples in the training set
train_x=features.iloc[train_list]
# the following variable records the features of examples in the test set
test_x=features.iloc[test_list]
# the following variable records the labels of examples in the training set
train_y=target.iloc[train_list]
# the following variable records the labels of examples in the test set
test_y=target.iloc[test_list]
print(len(train_x))
print(len(test_x))

326105


/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

260884
65221


## Ridge regression and grid search of parameter

In [37]:
Xtrain=np.array(train_x)
Xtest=np.array(test_x)
Ytrain=np.array(train_y)
Ytest=np.array(test_y)

alpha_list=np.linspace(20000,50000,51)
w_coef_list=[]
train_pred_list=[]
test_pred_list=[]
train_MAE_relative_list=[]
test_MAE_relative_list=[]
for alpha in alpha_list:
    model_ridge=Ridge(alpha=alpha,fit_intercept=True)
    model_ridge.fit(Xtrain,Ytrain)
    w_coef_list.append(np.array(model_ridge.coef_))
    train_pred = model_ridge.predict(Xtrain)
    test_pred = model_ridge.predict(Xtest)
    train_pred_list.append(train_pred)
    test_pred_list.append(test_pred)
    train_MAE_relative = MAE_relative(Ytrain,train_pred)
    test_MAE_relative = MAE_relative(Ytest,test_pred)
    train_MAE_relative_list.append(train_MAE_relative)
    test_MAE_relative_list.append(test_MAE_relative)
    
optimal_alpha_index=np.argmin(np.array(test_MAE_relative_list))
print("Optimal alpha is",alpha_list[optimal_alpha_index])
print("Train relative error:",train_MAE_relative_list[optimal_alpha_index])
print("Test relative error:",test_MAE_relative_list[optimal_alpha_index])

Optimal alpha is 39200.0
Train relative error: 0.48950823655244524
Test relative error: 0.4945261279101348


In [38]:
w_coef_list[optimal_alpha_index]

array([ 6.37009543e+02, -2.35478071e+03, -2.39169670e+00, -5.86354696e+01,
        3.96097290e+02, -1.11581136e+01, -9.58652139e+02, -4.40703395e+01,
       -1.93000556e+01,  5.85724118e+02,  3.59875278e+01,  1.53717112e+01,
       -3.32494347e+02,  1.12646984e+01,  2.53849388e+02,  2.05531123e+02,
       -2.42550479e+02,  1.81925738e+00, -6.21575724e+01,  1.95262770e+01,
       -5.78683413e+01,  7.61200488e+02, -6.56667770e+02,  2.96381405e+02,
       -4.31825203e+01, -1.54651608e+02,  1.25341892e+03, -9.78924006e+02,
       -3.31881493e+02,  5.73865759e+01,  6.08790785e-01,  6.78679851e+02,
        4.68961491e+03, -1.27055690e+02, -9.21160600e+02])

## Demo: K-fold cross validation

In [2]:
data_grouping=pd.read_csv("data_with_grouping_operations_IS.csv")
data_grouping.head(10)

,id,city,price,year,manufacturer,make,condition,cylinders,fuel,odometer,...,title_status_price_std,transmission_mean_price,transmission_median_price,transmission_price_std,type_mean_price,type_median_price,type_price_std,manufacturer_make_mean_price,manufacturer_make_median_price,manufacturer_make_price_std
0,1,"abilene, TX",31999,2012,ram,2500,NaN,0,diesel,NaN,...,11784.0,13736.0,10500.0,11630.0,NaN,NaN,NaN,25615.096,25999.0,12451.1170
1,3,"abilene, TX",6000,2002,gmc,sierra 1500,good,8,gas,195000.0,...,11784.0,13736.0,10500.0,11630.0,20980.0,19980.0,12580.0,19184.338,17893.0,11497.5840
2,4,"abilene, TX",37000,2012,chevrolet,3500,excellent,8,diesel,178000.0,...,11784.0,13736.0,10500.0,11630.0,20980.0,19980.0,12580.0,18962.380,13000.0,14690.9360
3,5,"abilene, TX",3700,2003,NaN,F150,fair,8,gas,269000.0,...,11784.0,13736.0,10500.0,11630.0,20980.0,19980.0,12580.0,NaN,NaN,NaN
4,6,"abilene, TX",19950,2013,ford,f-250,NaN,8,gas,116792.0,...,11784.0,13736.0,10500.0,11630.0,20980.0,19980.0,12580.0,20915.701,17999.0,13412.3080
5,7,"abilene, TX",19999,2006,ram,2500,NaN,0,diesel,NaN,...,11784.0,11940.0,7996.0,12970.0,NaN,NaN,NaN,25615.096,25999.0,12451.1170
6,8,"abilene, TX",33950,2015,ford,f-350,NaN,8,diesel,77350.0,...,11784.0,13736.0,10500.0,11630.0,20980.0,19980.0,12580.0,23164.252,19999.0,13095.8700
7,9,"abilene, TX",25950,2015,ford,f-350,NaN,8,gas,121030.0,...,11784.0,13736.0,10500.0,11630.0,19250.0,17000.0,13430.0,23164.252,19999.0,13095.8700
8,10,"abilene, TX",49950,2018,ram,5500,NaN,6,diesel,60491.0,...,11784.0,13736.0,10500.0,11630.0,19250.0,17000.0,13430.0,41587.695,43995.0,12020.8120
9,11,"abilene, TX",9700,2010,cadillac,srx luxury collection,good,8,gas,140000.0,...,11784.0,13736.0,10500.0,11630.0,13650.0,11496.0,10120.0,13365.096,12500.0,5756.3013


In [3]:
features_colname=['year','odometer','income_rank','median_income',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
       'type_onehot_10', 'type_onehot_11', 'type_onehot_12', 'type_onehot_13',
       'type_onehot_14','drive_onehot_1', 'drive_onehot_2', 'drive_onehot_3',
       'drive_onehot_4', 'title_status_mean_price','manufacturer_mean_price','manufacturer_make_mean_price','transmission_encoded', 'fuel_encoded']
data_clean=data_grouping[features_colname+["price"]]

# #fill nans in features
# features=features.fillna(features.mean())

#  drop nan
data_clean=data_clean.dropna(axis = 0, how ='any')
print(len(data_clean))
features=data_clean[features_colname]

# scaling
features['odometer']=(features['odometer']-features['odometer'].mean())/features['odometer'].std()
features['median_income']=(features['median_income']-features['median_income'].mean())/features['median_income'].std()
features['manufacturer_mean_price']=(features['manufacturer_mean_price']-features['manufacturer_mean_price'].mean())/features['manufacturer_mean_price'].std()
features['manufacturer_make_mean_price']=(features['manufacturer_make_mean_price']-features['manufacturer_make_mean_price'].mean())/features['manufacturer_make_mean_price'].std()
features['income_rank']=features['income_rank']/features['income_rank'].max()
target=data_clean["price"]

326248


/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zixin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [6]:
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(target):
    train_x=features.iloc[train_index]
    # the following variable records the features of examples in the test set
    test_x=features.iloc[test_index]
    # the following variable records the labels of examples in the training set
    train_y=target.iloc[train_index]
    # the following variable records the labels of examples in the test set
    test_y=target.iloc[test_index]